In [1]:
# !uv pip install numpy setuptools wheel cython
# !uv pip install pkuseg==0.0.25 --no-build-isolation
# !uv pip install faster-whisper

In [6]:
from faster_whisper import WhisperModel
import sys
import torch
import torchaudio as ta
from IPython.display import Audio, display, clear_output

c:\Users\Tseh\Documents\Files\Tests\env_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# STT

In [7]:
whisper_model = WhisperModel('medium', device="cuda", compute_type="float16")
# multilingual_model = ChatterboxMultilingualTTS.from_pretrained(device='cuda')
print("Models loaded.")

Models loaded.


In [10]:
# AUDIO_PROMPT_PATH = "neil/neil_audio.mp3"

segments, info = whisper_model.transcribe(
    "neil/neil_video.mp4", 
    beam_size=5, 
    language='en'
)

verified_segments = []

for i, segment in enumerate(segments):
    print(f"\n--- Segment {i+1} ---")
    print(f"Original: {segment.text}")
    
    while True:
        choice = input("\nAccept (a), Edit (e), or Skip (s)? ").strip().lower()
        
        if choice == 'a':
            verified_segments.append(segment.text)
            break
        elif choice == 'e':
            edited_text = input("Enter corrected text: ").strip()
            verified_segments.append(edited_text)
            break
        elif choice == 's':
            print("Skipped.")
            break
        else:
            print("Invalid choice. Use 'a', 'e', or 's'.")


--- Segment 1 ---
Original:  Do you believe in God? Me? A creator? Yeah, so I'm, the more I look at the universe, just

--- Segment 2 ---
Original:  the less convinced I am that there is something benevolent going on. So if you, if your concept

--- Segment 3 ---
Original:  of a creator is someone who's all-powerful and all-good, that's not an uncommon pairing

--- Segment 4 ---
Original:  of powers that you might describe to a creator. All-powerful and all-good. And I look at,

--- Segment 5 ---
Original:  disasters that afflict Earth and life on Earth. Volcanoes, hurricanes, tornadoes, earthquakes,

--- Segment 6 ---
Original:  disease, pestilence, congenital birth defects. You look at this list of ways that life is

--- Segment 7 ---
Original:  made miserable on Earth by natural causes. And I just ask, how do you deal with that?

--- Segment 8 ---
Original:  So philosophers rose up and said, if there is a God, God is either not all-powerful

--- Segment 9 ---
Original:  or not all-

In [11]:
verified_segments

[" Do you believe in God? Me? A creator? Yeah, so I'm, the more I look at the universe, just",
 ' the less convinced I am that there is something benevolent going on. So if you, if your concept',
 " of a creator is someone who's all-powerful and all-good, that's not an uncommon pairing",
 ' of powers that you might describe to a creator. All-powerful and all-good. And I look at,',
 ' disasters that afflict Earth and life on Earth. Volcanoes, hurricanes, tornadoes, earthquakes,',
 ' disease, pestilence, congenital birth defects. You look at this list of ways that life is',
 ' made miserable on Earth by natural causes. And I just ask, how do you deal with that?',
 ' So philosophers rose up and said, if there is a God, God is either not all-powerful',
 ' or not all-good. I have no problems if, as we probe the origins of things, we bump up',
 " into the bearded man. If that shows up, we're good to go. Okay? Not a problem. There's",
 ' just no evidence of it. And this is why religions are c

In [12]:
import re

def split_into_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s for s in sentences if s]

def group_sentences(segments, min_sentences=2, min_chars=100, max_chars=340):
    full_text = " ".join(segments)
    sentences = split_into_sentences(full_text)
    
    groups = []
    current_group = []
    
    for sentence in sentences:
        test_combined = " ".join(current_group + [sentence])
        
        if len(test_combined) > max_chars and len(current_group) >= min_sentences:
            groups.append(" ".join(current_group))
            current_group = [sentence]
        else:
            current_group.append(sentence)
            
            combined = " ".join(current_group)
            if (len(current_group) >= min_sentences and 
                len(combined) >= min_chars and 
                len(combined) <= max_chars):
                pass
    
    if current_group:
        combined = " ".join(current_group)
        if groups and len(combined) < min_chars:
            groups[-1] = groups[-1] + " " + combined
        else:
            groups.append(combined)
    
    return groups

tts_texts = group_sentences(verified_segments, min_sentences=2, min_chars=200, max_chars=310)

print(f"Created {len(tts_texts)} TTS groups:\n")
for i, text in enumerate(tts_texts, 1):
    print(f"--- Group {i} ({len(text)} chars) ---")
    print(text)
    print()

Created 6 TTS groups:

--- Group 1 (159 chars) ---
Do you believe in God? Me? A creator? Yeah, so I'm, the more I look at the universe, just  the less convinced I am that there is something benevolent going on.

--- Group 2 (255 chars) ---
So if you, if your concept  of a creator is someone who's all-powerful and all-good, that's not an uncommon pairing  of powers that you might describe to a creator. All-powerful and all-good. And I look at,  disasters that afflict Earth and life on Earth.

--- Group 3 (224 chars) ---
Volcanoes, hurricanes, tornadoes, earthquakes,  disease, pestilence, congenital birth defects. You look at this list of ways that life is  made miserable on Earth by natural causes. And I just ask, how do you deal with that?

--- Group 4 (283 chars) ---
So philosophers rose up and said, if there is a God, God is either not all-powerful  or not all-good. I have no problems if, as we probe the origins of things, we bump up  into the bearded man. If that shows up, we're goo

# Preparation for lipsync

In [1]:
from moviepy import VideoFileClip, AudioFileClip, concatenate_videoclips
from pydub import AudioSegment
import os
import shutil

In [ ]:
VIDEO_PATH = "neil/neil_video.mp4"
WORK_DIR = "neilstuff"
AUDIO_PATH = "neil"

os.makedirs(WORK_DIR, exist_ok=True)

wav_files = sorted(
    [f for f in os.listdir(AUDIO_PATH) if f.endswith('.wav')],
    key=lambda x: int(x.split('.')[0])
)

video = VideoFileClip(VIDEO_PATH)

current_time = 0
accumulated_duration = 0
segment_start = 0
part_index = 0
current_audio_segments = []

for i, wav_file in enumerate(wav_files):
    audio_seg = AudioSegment.from_wav(os.path.join(AUDIO_PATH, wav_file))
    duration = audio_seg.duration_seconds
    
    if accumulated_duration + duration > 60 and accumulated_duration > 0:
        video_part = video.subclipped(segment_start, current_time)
        video_part.write_videofile(
            f"{WORK_DIR}/video_{part_index}.mp4",
            codec="libx264",
            audio=False
        )
        video_part.close()
        
        combined_audio = sum(current_audio_segments)
        combined_audio.export(
            f"{WORK_DIR}/audio_{part_index}.wav",
            format="wav"
        )
        
        segment_start = current_time
        accumulated_duration = 0
        current_audio_segments = []
        part_index += 1
    
    current_audio_segments.append(audio_seg)
    accumulated_duration += duration
    current_time += duration

if segment_start < current_time:
    video_part = video.subclipped(segment_start, current_time)
    video_part.write_videofile(
        f"{WORK_DIR}/video_{part_index}.mp4",
        codec="libx264",
        audio=False
    )
    video_part.close()
    
    combined_audio = sum(current_audio_segments)
    combined_audio.export(
        f"{WORK_DIR}/audio_{part_index}.wav",
        format="wav"
    )

video.close()

shutil.make_archive(WORK_DIR, 'zip', WORK_DIR)

MoviePy - Building video neilstuff/video_0.mp4.
MoviePy - Writing video neilstuff/video_0.mp4



MoviePy - Done !
MoviePy - video ready neilstuff/video_0.mp4


'c:\\Users\\Tseh\\Documents\\Files\\Tests\\neilstuff.zip'

# Merge processed files

In [12]:
PROCESSED_DIR = "processes_video_parts"

clips = [VideoFileClip(f"{PROCESSED_DIR}/processed_part{i}.mp4") for i in range(2)]
final = concatenate_videoclips(clips, method="compose")
final.write_videofile("final_2mins.mp4", codec="libx264", audio_codec="aac")

final.close()
for clip in clips:
    clip.close()

MoviePy - Building video final_2mins.mp4.
MoviePy - Writing audio in final_2minsTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video final_2mins.mp4



MoviePy - Done !
MoviePy - video ready final_2mins.mp4
